***YASHASWEE NARAYAN -- 25/RCO/07***

# **1. Imports & Dataset Loading**

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist


# **2. Load & Preprocess MNIST**

In [2]:
# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize (0–255 → 0–1)
X_train = X_train / 255.0
X_test = X_test / 255.0

# Flatten 28x28 → 784
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train shape: (60000, 784)
Test shape: (10000, 784)


# **3. One-Hot Encoding (Labels)**

In [3]:
def one_hot(y, num_classes=10):
    one_hot_labels = np.zeros((y.size, num_classes))
    one_hot_labels[np.arange(y.size), y] = 1
    return one_hot_labels

y_train_oh = one_hot(y_train)
y_test_oh = one_hot(y_test)


# **4. Activation Functions**

In [4]:
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return Z > 0

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)


# **5. Loss Function (Categorical Cross-Entropy)**

In [5]:
def cross_entropy(y_true, y_pred):
    epsilon = 1e-9
    return -np.mean(np.sum(y_true * np.log(y_pred + epsilon), axis=1))


# **6. Initialize Network Parameters**

### **Architecture:**

### **Input: 784**

### **Hidden layer: 128**

### **Output: 10**

In [6]:
np.random.seed(42)

input_size = 784
hidden_size = 128
output_size = 10
learning_rate = 0.01

W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))

W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))


# **7. Forward Propagation**

In [7]:
def forward(X):
    Z1 = X @ W1 + b1
    A1 = relu(Z1)
    Z2 = A1 @ W2 + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2


# **8. Backpropagation**

In [8]:
def backward(X, y, Z1, A1, A2):
    global W1, b1, W2, b2

    m = X.shape[0]

    dZ2 = A2 - y
    dW2 = A1.T @ dZ2 / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = dZ2 @ W2.T
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = X.T @ dZ1 / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    # Update parameters
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2


# **9. Training Loop**

In [9]:
epochs = 20
batch_size = 64

for epoch in range(epochs):
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train_oh[i:i+batch_size]

        Z1, A1, Z2, A2 = forward(X_batch)
        backward(X_batch, y_batch, Z1, A1, A2)

    # Loss every epoch
    _, _, _, train_pred = forward(X_train)
    loss = cross_entropy(y_train_oh, train_pred)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")


Epoch 1/20, Loss: 1.0138
Epoch 2/20, Loss: 0.5271
Epoch 3/20, Loss: 0.4185
Epoch 4/20, Loss: 0.3715
Epoch 5/20, Loss: 0.3439
Epoch 6/20, Loss: 0.3244
Epoch 7/20, Loss: 0.3091
Epoch 8/20, Loss: 0.2962
Epoch 9/20, Loss: 0.2849
Epoch 10/20, Loss: 0.2746
Epoch 11/20, Loss: 0.2650
Epoch 12/20, Loss: 0.2560
Epoch 13/20, Loss: 0.2475
Epoch 14/20, Loss: 0.2396
Epoch 15/20, Loss: 0.2321
Epoch 16/20, Loss: 0.2250
Epoch 17/20, Loss: 0.2182
Epoch 18/20, Loss: 0.2118
Epoch 19/20, Loss: 0.2056
Epoch 20/20, Loss: 0.1998


# **10. Model Evaluation (Accuracy)**

In [10]:
def accuracy(X, y_true):
    _, _, _, preds = forward(X)
    predicted_labels = np.argmax(preds, axis=1)
    return np.mean(predicted_labels == y_true)

train_acc = accuracy(X_train, y_train)
test_acc = accuracy(X_test, y_test)

print("Training Accuracy:", train_acc)
print("Test Accuracy:", test_acc)


Training Accuracy: 0.9441333333333334
Test Accuracy: 0.9429
